# Create an Agent with a single Knowledge Base only

In this notebook you will learn how to create an Amazon Bedrock Agent that connects to a single Knowledge Bases for Amazon Bedrock to retrieve company data and complete tasks. 

The use case for this notebook is the Amazon Bedrock Documentation pages stored as PDFs. It will allow you to ask questions about Amazon Bedrock and get answers based on documents available in the Knowledge Base.

The steps to complete this notebook are:

1. Import the needed libraries
1. Create an S3 bucket and upload the data to it
1. Create the Knowledge Base for Amazon Bedrock and sync data to Knowledge Base
1. Create the Agent for Amazon Bedrock
1. Test the Agent
1. Clean up the resources created

## 1. Import the needed libraries

In [1]:
!pip install --upgrade -q opensearch-py
!pip install --upgrade -q requests-aws4auth
!pip install --upgrade -q boto3
!pip install --upgrade -q botocore
!pip install --upgrade -q awscli

In [2]:
import logging
import boto3
import time
import json
import uuid
import pprint
import os
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth

In [3]:
# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
s3_client = boto3.client('s3')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
open_search_serverless_client = boto3.client('opensearchserverless')

[2025-03-12 13:53:45,105] p16228 {credentials.py:1352} INFO - Found credentials in shared credentials file: ~/.aws/credentials


In [5]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-east-1', '535002878551')

In [6]:
# Generate random prefix for unique IAM roles, agent name and S3 Bucket and 
# assign variables
suffix = f"{region}-{account_id}"
agent_name = "bedrock-docs-kb-agents"
agent_alias_name = "bedrock-docs-alias"
bucket_name = f'{agent_name}-{suffix}'
bucket_arn = f"arn:aws:s3:::{bucket_name}"
bedrock_agent_bedrock_allow_policy_name = f"bda-bedrock-allow-{suffix}"
bedrock_agent_s3_allow_policy_name = f"bda-s3-allow-{suffix}"
bedrock_agent_kb_allow_policy_name = f"bda-kb-allow-{suffix}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_bedrock_docs'
kb_name = f'bedrock-docs-kb-{suffix}'
data_source_name = f'bedrock-docs-kb-docs-{suffix}'
kb_files_path = 'kb_documents'
kb_key = 'kb_documents'
kb_role_name = f'AmazonBedrockExecutionRoleForKnowledgeBase_bedrock_docs'
kb_bedrock_allow_policy_name = f"bd-kb-bedrock-allow-{suffix}"
kb_aoss_allow_policy_name = f"bd-kb-aoss-allow-{suffix}"
kb_s3_allow_policy_name = f"bd-kb-s3-allow-{suffix}"
kb_collection_name = f'bd-kbc-{suffix}'
# Select Amazon titan as the embedding model
embedding_model_arn = f'arn:aws:bedrock:{region}::foundation-model/amazon.titan-embed-text-v1'
kb_vector_index_name = "bedrock-knowledge-base-index"
kb_metadataField = 'bedrock-knowledge-base-metadata'
kb_textField = 'bedrock-knowledge-base-text'
kb_vectorField = 'bedrock-knowledge-base-vector'
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

# agent configuration
agent_instruction = """
You are an agent that support users working with Amazon Bedrock. You have access to Bedrock's documentation in a Knowledge Base
and you can Answer questions from this documentation. Only answer questions based on the documentation and reply with 
"There is no information about your question on the Amazon Bedrock Documentation at the moment, sorry! Do you want to ask another question?" 
If the answer to the question is not available in the documentation
"""

## 2. Upload the dataset to Amazon S3
Knowledge Bases for Amazon Bedrock, currently require data to reside in an Amazon S3 bucket. In this section we will create an Amazon S3 bucket and the files.

### 2.1 Create the Amazon S3 bucket

In [7]:
if region != 'us-east-1':
    s3_client.create_bucket(
        Bucket=bucket_name.lower(),
        CreateBucketConfiguration={'LocationConstraint': region}
    )
else:
    s3_client.create_bucket(Bucket=bucket_name)

### 2.2 Upload dataset to the Amazon S3 bucket

In [8]:
# Upload Knowledge Base files to this s3 bucket
for f in os.listdir(kb_files_path):
    if f.endswith(".pdf"):
        s3_client.upload_file(kb_files_path+'/'+f, bucket_name, kb_key+'/'+f)

## 3. Create a Knowledge Base for Amazon Bedrock

In this section we will go through all the steps to create and test a Knowledge Base. 

These are the steps to complete:
    
1. Create a Knowledge Base Role and its policies
1. Create a Vector Database
1. Create an OpenSearch Index
1. Create a Knowledge Base
1. Create a data source and attach to the recently created Knowledge Base
1. Ingest data to your knowledge Base

### 3.1 Create Knowledge Base Role and Policies

Let's first create IAM policies to allow our Knowledge Base to access Bedrock Titan Embedding Foundation model, Amazon OpenSearch Serverless and the S3 bucket with the Knowledge Base Files.

Once the policies are ready, we will create the Knowledge Base role

In [9]:
# Create IAM policies for KB to invoke embedding model
bedrock_kb_allow_fm_model_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                embedding_model_arn
            ]
        }
    ]
}

kb_bedrock_policy_json = json.dumps(bedrock_kb_allow_fm_model_policy_statement)

kb_bedrock_policy = iam_client.create_policy(
    PolicyName=kb_bedrock_allow_policy_name,
    PolicyDocument=kb_bedrock_policy_json
)

In [10]:
# Create IAM policies for KB to access OpenSearch Serverless
bedrock_kb_allow_aoss_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "aoss:APIAccessAll",
            "Resource": [
                f"arn:aws:aoss:{region}:{account_id}:collection/*"
            ]
        }
    ]
}


kb_aoss_policy_json = json.dumps(bedrock_kb_allow_aoss_policy_statement)

kb_aoss_policy = iam_client.create_policy(
    PolicyName=kb_aoss_allow_policy_name,
    PolicyDocument=kb_aoss_policy_json
)

In [11]:
kb_s3_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowKBAccessDocuments",
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                f"arn:aws:s3:::{bucket_name}/*",
                f"arn:aws:s3:::{bucket_name}"
            ],
            "Condition": {
                "StringEquals": {
                    "aws:ResourceAccount": f"{account_id}"
                }
            }
        }
    ]
}


kb_s3_json = json.dumps(kb_s3_allow_policy_statement)
kb_s3_policy = iam_client.create_policy(
    PolicyName=kb_s3_allow_policy_name,
    PolicyDocument=kb_s3_json
)

In [12]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
kb_role = iam_client.create_role(
    RoleName=kb_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=kb_role_name,
    PolicyArn=kb_bedrock_policy['Policy']['Arn']
)

iam_client.attach_role_policy(
    RoleName=kb_role_name,
    PolicyArn=kb_aoss_policy['Policy']['Arn']
)

iam_client.attach_role_policy(
    RoleName=kb_role_name,
    PolicyArn=kb_s3_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': '1a948bb5-81b7-4da7-86c9-c761bda8b26d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 12 Mar 2025 13:59:14 GMT',
   'x-amzn-requestid': '1a948bb5-81b7-4da7-86c9-c761bda8b26d',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

In [13]:
kb_role_arn = kb_role["Role"]["Arn"]
kb_role_arn

'arn:aws:iam::535002878551:role/AmazonBedrockExecutionRoleForKnowledgeBase_bedrock_docs'

### 3.2 Create Vector Database

Firt of all we have to create a vector store. In this section we will use Amazon OpenSerach Serverless.

Amazon OpenSearch Serverless is a serverless option in Amazon OpenSearch Service. As a developer, you can use OpenSearch Serverless to run petabyte-scale workloads without configuring, managing, and scaling OpenSearch clusters. You get the same interactive millisecond response times as OpenSearch Service with the simplicity of a serverless environment. Pay only for what you use by automatically scaling resources to provide the right amount of capacity for your application—without impacting data ingestion.


In [14]:
# Create OpenSearch Collection
security_policy_json = {
    "Rules": [
        {
            "ResourceType": "collection",
            "Resource":[
                f"collection/{kb_collection_name}"
            ]
        }
    ],
    "AWSOwnedKey": True
}
security_policy = open_search_serverless_client.create_security_policy(
    description='security policy of aoss collection',
    name=kb_collection_name,
    policy=json.dumps(security_policy_json),
    type='encryption'
)

In [15]:
network_policy_json = [
  {
    "Rules": [
      {
        "Resource": [
          f"collection/{kb_collection_name}"
        ],
        "ResourceType": "dashboard"
      },
      {
        "Resource": [
          f"collection/{kb_collection_name}"
        ],
        "ResourceType": "collection"
      }
    ],
    "AllowFromPublic": True
  }
]

network_policy = open_search_serverless_client.create_security_policy(
    description='network policy of aoss collection',
    name=kb_collection_name,
    policy=json.dumps(network_policy_json),
    type='network'
)

In [16]:
response = sts_client.get_caller_identity()
current_role = response['Arn']
current_role

'arn:aws:iam::535002878551:root'

In [17]:
data_policy_json = [
  {
    "Rules": [
      {
        "Resource": [
          f"collection/{kb_collection_name}"
        ],
        "Permission": [
          "aoss:DescribeCollectionItems",
          "aoss:CreateCollectionItems",
          "aoss:UpdateCollectionItems",
          "aoss:DeleteCollectionItems"
        ],
        "ResourceType": "collection"
      },
      {
        "Resource": [
          f"index/{kb_collection_name}/*"
        ],
        "Permission": [
            "aoss:CreateIndex",
            "aoss:DeleteIndex",
            "aoss:UpdateIndex",
            "aoss:DescribeIndex",
            "aoss:ReadDocument",
            "aoss:WriteDocument"
        ],
        "ResourceType": "index"
      }
    ],
    "Principal": [
        kb_role_arn,
        f"arn:aws:sts::{account_id}:assumed-role/Admin/*",
        current_role
    ],
    "Description": ""
  }
]

data_policy = open_search_serverless_client.create_access_policy(
    description='data access policy for aoss collection',
    name=kb_collection_name,
    policy=json.dumps(data_policy_json),
    type='data'
)


In [18]:
opensearch_collection_response = open_search_serverless_client.create_collection(
    description='OpenSearch collection for Amazon Bedrock Knowledge Base',
    name=kb_collection_name,
    standbyReplicas='DISABLED',
    type='VECTORSEARCH'
)
opensearch_collection_response

{'createCollectionDetail': {'id': 'd25j2ytfucz87t87l5v1',
  'name': 'bd-kbc-us-east-1-535002878551',
  'status': 'CREATING',
  'type': 'VECTORSEARCH',
  'description': 'OpenSearch collection for Amazon Bedrock Knowledge Base',
  'arn': 'arn:aws:aoss:us-east-1:535002878551:collection/d25j2ytfucz87t87l5v1',
  'kmsKeyArn': 'auto',
  'standbyReplicas': 'DISABLED',
  'createdDate': 1741788071222,
  'lastModifiedDate': 1741788071222},
 'ResponseMetadata': {'RequestId': '1726414a-0d7a-4812-81be-cc69eea16870',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1726414a-0d7a-4812-81be-cc69eea16870',
   'date': 'Wed, 12 Mar 2025 14:01:11 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '394',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [19]:
collection_arn = opensearch_collection_response["createCollectionDetail"]["arn"]
collection_arn

'arn:aws:aoss:us-east-1:535002878551:collection/d25j2ytfucz87t87l5v1'

In [20]:
# wait for collection creation
response = open_search_serverless_client.batch_get_collection(names=[kb_collection_name])
# Periodically check collection status
while (response['collectionDetails'][0]['status']) == 'CREATING':
    print('Creating collection...')
    time.sleep(30)
    response = open_search_serverless_client.batch_get_collection(names=[kb_collection_name])
print('\nCollection successfully created:')
print(response["collectionDetails"])
# Extract the collection endpoint from the response
host = (response['collectionDetails'][0]['collectionEndpoint'])
final_host = host.replace("https://", "")
final_host

Creating collection...
Creating collection...
Creating collection...
Creating collection...
Creating collection...
Creating collection...

Collection successfully created:
[{'id': 'd25j2ytfucz87t87l5v1', 'name': 'bd-kbc-us-east-1-535002878551', 'status': 'ACTIVE', 'type': 'VECTORSEARCH', 'description': 'OpenSearch collection for Amazon Bedrock Knowledge Base', 'arn': 'arn:aws:aoss:us-east-1:535002878551:collection/d25j2ytfucz87t87l5v1', 'kmsKeyArn': 'auto', 'standbyReplicas': 'DISABLED', 'createdDate': 1741788071222, 'lastModifiedDate': 1741788308190, 'collectionEndpoint': 'https://d25j2ytfucz87t87l5v1.us-east-1.aoss.amazonaws.com', 'dashboardEndpoint': 'https://d25j2ytfucz87t87l5v1.us-east-1.aoss.amazonaws.com/_dashboards'}]


'd25j2ytfucz87t87l5v1.us-east-1.aoss.amazonaws.com'

### 3.3 - Create OpenSearch Index

Let's now create a vector index to index our data


In [21]:
credentials = boto3.Session().get_credentials()
service = 'aoss'
awsauth = AWS4Auth(
    credentials.access_key, 
    credentials.secret_key,
    region, 
    service, 
    session_token=credentials.token
)

# Build the OpenSearch client
open_search_client = OpenSearch(
    hosts=[{'host': final_host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)
# It can take up to a minute for data access rules to be enforced
time.sleep(45)
index_body = {
    "settings": {
        "index.knn": True,
        "number_of_shards": 1,
        "knn.algo_param.ef_search": 512,
        "number_of_replicas": 0,
    },
    "mappings": {
        "properties": {}
    }
}

index_body["mappings"]["properties"][kb_vectorField] = {
    "type": "knn_vector",
    "dimension": 1536,
    "method": {
         "name": "hnsw",
         "engine": "faiss"
    },
}

index_body["mappings"]["properties"][kb_textField] = {
    "type": "text"
}

index_body["mappings"]["properties"][kb_metadataField] = {
    "type": "text"
}

# Create index
response = open_search_client.indices.create(kb_vector_index_name, body=index_body)
print('\nCreating index:')
print(response)

[2025-03-12 14:06:15,372] p16228 {credentials.py:1352} INFO - Found credentials in shared credentials file: ~/.aws/credentials
[2025-03-12 14:07:01,460] p16228 {base.py:258} INFO - PUT https://d25j2ytfucz87t87l5v1.us-east-1.aoss.amazonaws.com:443/bedrock-knowledge-base-index [status:200 request:1.077s]



Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'bedrock-knowledge-base-index'}


### 3.5 - Create Knowledge Base
Now that we have the Vector database available in OpenSearch Serverless, let's create a Knowledge Base and associate it with the OpenSearch DB

In [22]:
storage_configuration = {
    'opensearchServerlessConfiguration': {
        'collectionArn': collection_arn, 
        'fieldMapping': {
            'metadataField': kb_metadataField,
            'textField': kb_textField,
            'vectorField': kb_vectorField
        },
        'vectorIndexName': kb_vector_index_name
    },
    'type': 'OPENSEARCH_SERVERLESS'
}

In [23]:
# Creating the knowledge base
try:
    # ensure the index is created and available
    time.sleep(45)
    kb_obj = bedrock_agent_client.create_knowledge_base(
        name=kb_name, 
        description='KB that contains the bedrock documentation',
        roleArn=kb_role_arn,
        knowledgeBaseConfiguration={
            'type': 'VECTOR',  # Corrected type
            'vectorKnowledgeBaseConfiguration': {
                'embeddingModelArn': embedding_model_arn
            }
        },
        storageConfiguration=storage_configuration
    )

    # Pretty print the response
    pprint.pprint(kb_obj)

except Exception as e:
    print(f"Error occurred: {e}")

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '1013',
                                      'content-type': 'application/json',
                                      'date': 'Wed, 12 Mar 2025 14:08:24 GMT',
                                      'x-amz-apigw-id': 'HURFfFZaIAMEKjQ=',
                                      'x-amzn-requestid': 'cae82bc2-8632-484e-a909-98f7d86ae754',
                                      'x-amzn-trace-id': 'Root=1-67d19555-52ceb0eb272e1a4a49ddd983'},
                      'HTTPStatusCode': 202,
                      'RequestId': 'cae82bc2-8632-484e-a909-98f7d86ae754',
                      'RetryAttempts': 0},
 'knowledgeBase': {'createdAt': datetime.datetime(2025, 3, 12, 14, 8, 22, 28413, tzinfo=tzutc()),
                   'description': 'KB that contains the bedrock documentation',
                   'knowledgeBaseArn': 'arn:aws:bedrock:us-east-1:535002878551:knowledge-base/LDRESE

In [24]:
# Define the S3 configuration for your data source
s3_configuration = {
    'bucketArn': bucket_arn,
    'inclusionPrefixes': [kb_key]  
}

# Define the data source configuration
data_source_configuration = {
    's3Configuration': s3_configuration,
    'type': 'S3'
}

knowledge_base_id = kb_obj["knowledgeBase"]["knowledgeBaseId"]
knowledge_base_arn = kb_obj["knowledgeBase"]["knowledgeBaseArn"]

chunking_strategy_configuration = {
    "chunkingStrategy": "FIXED_SIZE",
    "fixedSizeChunkingConfiguration": {
        "maxTokens": 512,
        "overlapPercentage": 20
    }
}

# Create the data source
try:
    # ensure that the KB is created and available
    time.sleep(45)
    data_source_response = bedrock_agent_client.create_data_source(
        knowledgeBaseId=knowledge_base_id,
        name=data_source_name,
        description='DataSource for the bedrock documentation',
        dataSourceConfiguration=data_source_configuration,
        vectorIngestionConfiguration = {
            "chunkingConfiguration": chunking_strategy_configuration
        }
    )

    # Pretty print the response
    pprint.pprint(data_source_response)

except Exception as e:
    print(f"Error occurred: {e}")


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '661',
                                      'content-type': 'application/json',
                                      'date': 'Wed, 12 Mar 2025 14:09:24 GMT',
                                      'x-amz-apigw-id': 'HURPREsAoAMEiFQ=',
                                      'x-amzn-requestid': '17c74b11-e1b4-48b7-b997-850dce2c4099',
                                      'x-amzn-trace-id': 'Root=1-67d19594-1a578af4556fa4f4486dfb13'},
                      'HTTPStatusCode': 200,
                      'RequestId': '17c74b11-e1b4-48b7-b997-850dce2c4099',
                      'RetryAttempts': 0},
 'dataSource': {'createdAt': datetime.datetime(2025, 3, 12, 14, 9, 24, 571169, tzinfo=tzutc()),
                'dataDeletionPolicy': 'DELETE',
                'dataSourceConfiguration': {'s3Configuration': {'bucketArn': 'arn:aws:s3:::bedrock-docs-kb-agents-us-east-1-53500287855

### 3.6 - Start ingestion job

Once the Knowledge Base and Data Source are created, we can start the ingestion job. During the ingestion job, Knowledge Base will fetch the documents in the data source, pre-process it to extract text, chunk it based on the chunking size provided, create embeddings of each chunk and then write it to the vector database, in this case Amazon OpenSource Serverless.


In [25]:
# Start an ingestion job
data_source_id = data_source_response["dataSource"]["dataSourceId"]
start_job_response = bedrock_agent_client.start_ingestion_job(
    knowledgeBaseId=knowledge_base_id, 
    dataSourceId=data_source_id
)


## 4. Create Agent

We will now create the Agent and associate the Knowledge Base to it. To do so we need to: 
1. Create Agent IAM role and policies
1. Create Agent
1. Associate Agent to Knowledge Base
1. Prepare Agent

### 4.1 - Create Agent IAM role and policies
First we need to create the agent policies that allow bedrock model invocation and Knowledge Base retrieval

In [26]:
# Create IAM policies for agent
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{model_id}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=bedrock_agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)

In [27]:
bedrock_agent_kb_retrival_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:Retrieve"
            ],
            "Resource": [
                knowledge_base_arn
            ]
        }
    ]
}
bedrock_agent_kb_json = json.dumps(bedrock_agent_kb_retrival_policy_statement)
agent_kb_schema_policy = iam_client.create_policy(
    PolicyName=bedrock_agent_kb_allow_policy_name,
    Description=f"Policy to allow agent to retrieve documents from knowledge base.",
    PolicyDocument=bedrock_agent_kb_json
)


In [28]:

# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)


iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_kb_schema_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': '23159f28-4712-4305-9cc7-e3d485ebb307',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 12 Mar 2025 14:10:14 GMT',
   'x-amzn-requestid': '23159f28-4712-4305-9cc7-e3d485ebb307',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

### 4.2 - Create Agent
Once the needed IAM role is created, we can use the bedrock agent client to create a new agent. To do so we use the create_agent function. It requires an agent name, underline foundation model and instruction. You can also provide an agent description. Note that the agent created is not yet prepared. We will focus on preparing the agent and then using it to invoke actions and use other APIs

In [29]:
# Create Agent
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description="Agent supporting Amazon Bedrock Developers.",
    idleSessionTTLInSeconds=1800,
    foundationModel=model_id,
    instruction=agent_instruction,
)

Let's now store the agent id in a local variable to use it on the next steps

In [30]:
agent_id = response['agent']['agentId']
agent_id

'CNZOEAJYSZ'

### 4.3 - Associate agent to the Knowledge Base
Next, we need to associate the agent created with the Knowledge Base for the Bedrock documentation

In [31]:
agent_kb_description = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    description=f'Use the information in the {kb_name} knowledge base to provide accurate responses to the questions about Amazon Bedrock.',
    knowledgeBaseId=knowledge_base_id 
)

### 4.4 - Prepare Agent

Let's create a DRAFT version of the agent that can be used for internal testing.


In [32]:
agent_prepare = bedrock_agent_client.prepare_agent(agentId=agent_id)
agent_prepare

{'ResponseMetadata': {'RequestId': '5a1f319f-6792-4f18-980e-12655e4378ee',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Wed, 12 Mar 2025 14:10:48 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': '5a1f319f-6792-4f18-980e-12655e4378ee',
   'x-amz-apigw-id': 'HURcZEY8IAMED4Q=',
   'x-amzn-trace-id': 'Root=1-67d195e8-0aa418b17a9b173d585427dc'},
  'RetryAttempts': 0},
 'agentId': 'CNZOEAJYSZ',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2025, 3, 12, 14, 10, 48, 789608, tzinfo=tzutc())}

## 5 - Testing Agent

Now that we have our agent, let's invoke it to test if it is providing correct information about Amazon Bedrock. To do so, let's first create an Agent Alias

In [33]:
# Pause to make sure agent is prepared
time.sleep(30)
agent_alias = bedrock_agent_client.create_agent_alias(
    agentId=agent_id,
    agentAliasName=agent_alias_name
)
# Pause to make sure agent alias is ready
time.sleep(30)

In [34]:
agent_alias

{'ResponseMetadata': {'RequestId': '42d85dc6-8a95-4256-b174-d96d0ebe59ac',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Wed, 12 Mar 2025 14:11:45 GMT',
   'content-type': 'application/json',
   'content-length': '344',
   'connection': 'keep-alive',
   'x-amzn-requestid': '42d85dc6-8a95-4256-b174-d96d0ebe59ac',
   'x-amz-apigw-id': 'HURlPGG-oAMEKSQ=',
   'x-amzn-trace-id': 'Root=1-67d19621-6751bf816bf7d96331f909d7'},
  'RetryAttempts': 0},
 'agentAlias': {'agentAliasArn': 'arn:aws:bedrock:us-east-1:535002878551:agent-alias/CNZOEAJYSZ/ILSFPV2ELR',
  'agentAliasId': 'ILSFPV2ELR',
  'agentAliasName': 'bedrock-docs-alias',
  'agentAliasStatus': 'CREATING',
  'agentId': 'CNZOEAJYSZ',
  'createdAt': datetime.datetime(2025, 3, 12, 14, 11, 45, 250050, tzinfo=tzutc()),
  'routingConfiguration': [{}],
  'updatedAt': datetime.datetime(2025, 3, 12, 14, 11, 45, 250050, tzinfo=tzutc())}}

Now that we've created the agent, let's use the bedrock-agent-runtime client to invoke this agent and get the information from the Knowledge base

In [35]:
# Extract the agentAliasId from the response
agent_alias_id = agent_alias['agentAlias']['agentAliasId']

## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = True
end_session:bool = False

# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="How can I evaluate models on Bedrock?",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))


[2025-03-12 14:12:44,399] p16228 {2021949151.py:19} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/vnd.amazon.eventstream',
                                      'date': 'Wed, 12 Mar 2025 14:12:44 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '10f4c138-ff4c-11ef-9bfa-acde48001122',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '58232aee-3677-4b27-9a27-058278a84250'},
                      'HTTPStatusCode': 200,
                      'RequestId': '58232aee-3677-4b27-9a27-058278a84250',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x10b142040>,
 'contentType': 'application/json',
 'sessionId': '10f4c138-ff4c-11ef-9bfa-acde48001122'}


In [39]:
import json
from datetime import datetime

def default_serializer(obj):
    if isinstance(obj, datetime):
        return obj.isoformat()
    raise TypeError(f'Object of type {obj.__class__.__name__} is not JSON serializable')

# %%time
event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2, default=default_serializer))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2025-03-12 14:17:23,003] p16228 {2933599886.py:20} INFO - {
  "agentAliasId": "ILSFPV2ELR",
  "agentId": "CNZOEAJYSZ",
  "agentVersion": "1",
  "callerChain": [
    {
      "agentAliasArn": "arn:aws:bedrock:us-east-1:535002878551:agent-alias/CNZOEAJYSZ/ILSFPV2ELR"
    }
  ],
  "eventTime": "2025-03-12T14:12:45.019299+00:00",
  "sessionId": "10f4c138-ff4c-11ef-9bfa-acde48001122",
  "trace": {
    "orchestrationTrace": {
      "modelInvocationInput": {
        "inferenceConfiguration": {
          "maximumLength": 2048,
          "stopSequences": [
            "\n\nHuman:"
          ],
          "temperature": 0.0,
          "topK": 250,
          "topP": 1.0
        },
        "text": "{\"system\":\"You are a question answering agent. I will provide you with a set of search results. The user will provide you with a question. Your job is to answer the user's question using only information from the search results. If the search results do not contain information that can answer the ques

In [ ]:
# %%time
# event_stream = agentResponse['completion']
# try:
#     for event in event_stream:        
#         if 'chunk' in event:
#             data = event['chunk']['bytes']
#             logger.info(f"Final answer ->\n{data.decode('utf8')}")
#             agent_answer = data.decode('utf8')
#             end_event_received = True
#             # End event indicates that the request finished successfully
#         elif 'trace' in event:
#             logger.info(json.dumps(event['trace'], indent=2))
#         else:
#             raise Exception("unexpected event.", event)
# except Exception as e:
#     raise Exception("unexpected event.", e)

Exception: ('unexpected event.', TypeError('Object of type datetime is not JSON serializable'))

In [40]:
# And here is the response if you just want to see agent's reply
print(agent_answer)

You can evaluate models on Amazon Bedrock by creating a model evaluation job. There are two types of model evaluation jobs you can create:

1. Automatic model evaluation: This allows you to evaluate the responses from a single model using recommended metrics and built-in or custom prompt datasets. You can have up to 10 automatic model evaluation jobs in progress per AWS region.

2. Human worker-based model evaluation: This allows you to evaluate up to two models using human workers. You can define custom metrics and provide your own prompt dataset. You can have up to 10 human worker-based model evaluation jobs in progress per AWS region. 
To create a model evaluation job, you need to have access to the model(s) in Amazon Bedrock, an Amazon Bedrock service role with the required permissions, and any custom prompt datasets must have the required CORS permissions on the S3 bucket. You can create the service role during the job setup process if needed.


In [41]:
def simple_agent_invoke(input_text, agent_id, agent_alias_id, session_id=None, enable_trace=False, end_session=False):
    if session_id is None:
        session_id:str = str(uuid.uuid1())

    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=input_text,
        agentId=agent_id,
        agentAliasId=agent_alias_id, 
        sessionId=session_id,
        enableTrace=enable_trace, 
        endSession= end_session
    )
    logger.info(pprint.pprint(agentResponse))
    
    agent_answer = ''
    event_stream = agentResponse['completion']
    try:
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)
    return agent_answer

In [42]:
simple_agent_invoke("what is bedrock provisioned throughput?", agent_id, agent_alias_id, session_id)

[2025-03-12 14:18:22,818] p16228 {321621934.py:13} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/vnd.amazon.eventstream',
                                      'date': 'Wed, 12 Mar 2025 14:18:22 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '10f4c138-ff4c-11ef-9bfa-acde48001122',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '3cfb6dec-58e7-4221-9b30-e143622b6873'},
                      'HTTPStatusCode': 200,
                      'RequestId': '3cfb6dec-58e7-4221-9b30-e143622b6873',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x10b4e9af0>,
 'contentType': 'application/json',
 'sessionId': '10f4c138-ff4c-11ef-9bfa-acde48001122'}


[2025-03-12 14:18:31,439] p16228 {321621934.py:21} INFO - Final answer ->
Amazon Bedrock provisioned throughput allows you to purchase a guaranteed level of throughput for a specific foundation model or custom model. By purchasing provisioned throughput, you can ensure high availability and consistent performance for your model inference workloads. Provisioned throughput is measured in model units (MUs) and you can choose between different commitment levels (no commitment, 1 month, or 6 months). 
With provisioned throughput, you specify the model version that will remain unchanged for the entire commitment term. This allows you to lock in a specific model version and avoid disruptions from model updates or end-of-life cycles. Provisioned throughput is supported for select foundation models from providers like Anthropic, Stable Diffusion, and Amazon's own models like Amazon Titan.


"Amazon Bedrock provisioned throughput allows you to purchase a guaranteed level of throughput for a specific foundation model or custom model. By purchasing provisioned throughput, you can ensure high availability and consistent performance for your model inference workloads. Provisioned throughput is measured in model units (MUs) and you can choose between different commitment levels (no commitment, 1 month, or 6 months). \nWith provisioned throughput, you specify the model version that will remain unchanged for the entire commitment term. This allows you to lock in a specific model version and avoid disruptions from model updates or end-of-life cycles. Provisioned throughput is supported for select foundation models from providers like Anthropic, Stable Diffusion, and Amazon's own models like Amazon Titan."

In [43]:
simple_agent_invoke("what are the components of a Bedrock Guardrail?", agent_id, agent_alias_id, session_id)

[2025-03-12 14:18:41,781] p16228 {321621934.py:13} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/vnd.amazon.eventstream',
                                      'date': 'Wed, 12 Mar 2025 14:18:41 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '10f4c138-ff4c-11ef-9bfa-acde48001122',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': 'f9807fbc-3ec5-44b0-9ac9-f2104bc89761'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'f9807fbc-3ec5-44b0-9ac9-f2104bc89761',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x10b4e9a90>,
 'contentType': 'application/json',
 'sessionId': '10f4c138-ff4c-11ef-9bfa-acde48001122'}


[2025-03-12 14:18:51,919] p16228 {321621934.py:21} INFO - Final answer ->
The main components that make up an Amazon Bedrock Guardrail are:

1. Content filters: These allow you to adjust filter strengths to filter input prompts or model responses containing harmful content. They include word filters to block undesirable words, phrases, and profanity, as well as sensitive information filters to block or mask personally identifiable information (PII) and use regular expressions to define and block sensitive patterns.

2. Denied topics: You can define topics to block and provide sample phrases to include. These topics are described in a precise and concise manner without using instructions or negative language.

3. Messages: You can configure the messages to be returned to the user if a user input or model response violates the policies defined in the guardrail.


'The main components that make up an Amazon Bedrock Guardrail are:\n\n1. Content filters: These allow you to adjust filter strengths to filter input prompts or model responses containing harmful content. They include word filters to block undesirable words, phrases, and profanity, as well as sensitive information filters to block or mask personally identifiable information (PII) and use regular expressions to define and block sensitive patterns.\n\n2. Denied topics: You can define topics to block and provide sample phrases to include. These topics are described in a precise and concise manner without using instructions or negative language.\n\n3. Messages: You can configure the messages to be returned to the user if a user input or model response violates the policies defined in the guardrail.'

In [ ]:
simple_agent_invoke("what are the components of a Bedrock Guardrail?", agent_id, agent_alias_id, session_id)

## 6 - Clean up (Optional)

The next steps are optional and demonstrate how to delete our agent. To delete the agent we need to:

1. delete agent alias
1. delete agent
1. delete the knowledge base
1. delete the OpenSearch Serverless vector store
1. empty created s3 bucket
1. delete s3 bucket


In [44]:
agent_alias_deletion = bedrock_agent_client.delete_agent_alias(
    agentId=agent_id,
    agentAliasId=agent_alias['agentAlias']['agentAliasId']
)

In [45]:
agent_deletion = bedrock_agent_client.delete_agent(
    agentId=agent_id
)

In [46]:
# Empty and delete S3 Bucket

objects = s3_client.list_objects(Bucket=bucket_name)  
if 'Contents' in objects:
    for obj in objects['Contents']:
        s3_client.delete_object(Bucket=bucket_name, Key=obj['Key']) 
s3_client.delete_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': '7P5DCN47KG2VWC8P',
  'HostId': 'xx+czXOgRTwUKS8amXOWNAk4xtFcAwHlizJ2zaETLuZEHy73TO0WjMipxgdvbitHCxa+VW04aY0=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'xx+czXOgRTwUKS8amXOWNAk4xtFcAwHlizJ2zaETLuZEHy73TO0WjMipxgdvbitHCxa+VW04aY0=',
   'x-amz-request-id': '7P5DCN47KG2VWC8P',
   'date': 'Wed, 12 Mar 2025 14:19:40 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [47]:
# Delete IAM Roles and policies and Knowledge Base files
for policy in [
    agent_bedrock_policy, 
    agent_kb_schema_policy,
    kb_bedrock_policy,
    kb_aoss_policy,
    kb_s3_policy
]:
    response = iam_client.list_entities_for_policy(
        PolicyArn=policy['Policy']['Arn'],
        EntityFilter='Role'
    )

    for role in response['PolicyRoles']:
        iam_client.detach_role_policy(
            RoleName=role['RoleName'], 
            PolicyArn=policy['Policy']['Arn']
        )

    iam_client.delete_policy(
        PolicyArn=policy['Policy']['Arn']
    )

    

for role_name in [
    agent_role_name, 
    kb_role_name
]:
    try: 
        iam_client.delete_role(
            RoleName=role_name
        )
    except Exception as e:
        print(e)
        print("couldn't delete role", role_name)
        
    
try:

    open_search_serverless_client.delete_collection(
        id=opensearch_collection_response["createCollectionDetail"]["id"]
    )

    open_search_serverless_client.delete_access_policy(
          name=kb_collection_name,
          type='data'
    )    

    open_search_serverless_client.delete_security_policy(
          name=kb_collection_name,
          type='network'
    )   

    open_search_serverless_client.delete_security_policy(
          name=kb_collection_name,
          type='encryption'
    )    
    bedrock_agent_client.delete_knowledge_base(
        knowledgeBaseId=knowledge_base_id
    )
except Exception as e:
    print(e)

## Conclusion

We have now experimented with using boto3 SDK to create, invoke and delete an agent having a single KB connected to it.
## Take aways

Adapt this notebook to create new agents for your application

## Thank You